In [ ]:
# ROC Curves and Calibration Analysis
def plot_roc_curves_and_calibration(model_results, y_true, title_suffix=""):
    """Create comprehensive ROC curves and calibration plots for relapse prediction."""
    from sklearn.metrics import roc_curve, auc, calibration_curve, brier_score_loss
    from sklearn.calibration import calibration_curve
    
    # Set up the plotting layout
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Colors for different models
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FED766']
    
    # 1. ROC Curves (Top Left)
    ax_roc = axes[0, 0]
    
    for i, (model_name, results) in enumerate(model_results.items()):
        y_pred_proba = results['y_pred_proba'][:, 1]  # Probability of positive class
        
        fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        
        ax_roc.plot(fpr, tpr, color=colors[i], lw=2, 
                   label=f'{model_name} (AUC = {roc_auc:.3f})')
    
    # Plot random classifier line
    ax_roc.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', alpha=0.7, label='Random')
    
    ax_roc.set_xlim([0.0, 1.0])
    ax_roc.set_ylim([0.0, 1.05])
    ax_roc.set_xlabel('False Positive Rate', fontsize=12)
    ax_roc.set_ylabel('True Positive Rate', fontsize=12)
    ax_roc.set_title(f'ROC Curves - Relapse Prediction{title_suffix}', fontsize=14, pad=20)
    ax_roc.legend(loc="lower right", fontsize=10)
    ax_roc.grid(alpha=0.3)
    
    # 2. Precision-Recall Curves (Top Right)
    from sklearn.metrics import precision_recall_curve, average_precision_score
    ax_pr = axes[0, 1]
    
    for i, (model_name, results) in enumerate(model_results.items()):
        y_pred_proba = results['y_pred_proba'][:, 1]
        
        precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
        avg_precision = average_precision_score(y_true, y_pred_proba)
        
        ax_pr.plot(recall, precision, color=colors[i], lw=2,
                  label=f'{model_name} (AP = {avg_precision:.3f})')
    
    # Baseline (random classifier for imbalanced data)
    baseline = np.sum(y_true) / len(y_true)
    ax_pr.axhline(y=baseline, color='gray', linestyle='--', alpha=0.7, 
                 label=f'Baseline (AP = {baseline:.3f})')
    
    ax_pr.set_xlim([0.0, 1.0])
    ax_pr.set_ylim([0.0, 1.05])
    ax_pr.set_xlabel('Recall', fontsize=12)
    ax_pr.set_ylabel('Precision', fontsize=12)
    ax_pr.set_title(f'Precision-Recall Curves{title_suffix}', fontsize=14, pad=20)
    ax_pr.legend(loc="lower left", fontsize=10)
    ax_pr.grid(alpha=0.3)
    
    # 3. Calibration Plot (Bottom Left)
    ax_cal = axes[1, 0]
    
    for i, (model_name, results) in enumerate(model_results.items()):
        y_pred_proba = results['y_pred_proba'][:, 1]
        
        fraction_of_positives, mean_predicted_value = calibration_curve(
            y_true, y_pred_proba, n_bins=10, normalize=False
        )
        
        brier_score = brier_score_loss(y_true, y_pred_proba)
        
        ax_cal.plot(mean_predicted_value, fraction_of_positives, "s-", 
                   color=colors[i], lw=2, label=f'{model_name} (Brier = {brier_score:.3f})')
    
    # Perfect calibration line
    ax_cal.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    
    ax_cal.set_xlim([0.0, 1.0])
    ax_cal.set_ylim([0.0, 1.0])
    ax_cal.set_xlabel('Mean Predicted Probability', fontsize=12)
    ax_cal.set_ylabel('Fraction of Positives', fontsize=12)
    ax_cal.set_title(f'Calibration Plot (Reliability Diagram){title_suffix}', fontsize=14, pad=20)
    ax_cal.legend(loc="lower right", fontsize=10)
    ax_cal.grid(alpha=0.3)
    
    # 4. Performance Summary Table (Bottom Right)
    ax_table = axes[1, 1]
    ax_table.axis('off')
    
    # Create performance summary
    summary_data = []
    metrics = ['Accuracy', 'ROC AUC', 'F1 Score', 'Brier Score']
    
    for model_name, results in model_results.items():
        y_pred_proba = results['y_pred_proba'][:, 1]
        brier = brier_score_loss(y_true, y_pred_proba)
        
        row = [
            f"{results['accuracy']:.3f}",
            f"{results['roc_auc']:.3f}",
            f"{results['f1_score']:.3f}",
            f"{brier:.3f}"
        ]
        summary_data.append(row)
    
    # Create table
    table = ax_table.table(cellText=summary_data,
                          rowLabels=list(model_results.keys()),
                          colLabels=metrics,
                          cellLoc='center',
                          loc='center',
                          colWidths=[0.2, 0.2, 0.2, 0.2])
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    # Style the table
    for i in range(len(metrics)):
        table[(0, i)].set_facecolor('#E8E8E8')
        table[(0, i)].set_text_props(weight='bold')
    
    for i in range(len(model_results)):
        table[(i+1, -1)].set_facecolor('#F5F5F5')
    
    ax_table.set_title(f'Performance Summary{title_suffix}', fontsize=14, pad=20)
    
    plt.suptitle(f'Comprehensive Model Evaluation - Relapse Prediction{title_suffix}', 
                 fontsize=16, y=0.98)
    plt.tight_layout()
    plt.show()
    
    # Print detailed analysis
    print(f"\\n{'='*80}")
    print(f"📊 DETAILED PERFORMANCE ANALYSIS{title_suffix}")
    print(f"{'='*80}")
    
    best_model = max(model_results.items(), key=lambda x: x[1]['roc_auc'])
    print(f"🏆 Best Model by ROC AUC: {best_model[0]} (AUC = {best_model[1]['roc_auc']:.3f})")
    
    print(f"\\n📈 Model Rankings:")
    sorted_models = sorted(model_results.items(), key=lambda x: x[1]['roc_auc'], reverse=True)
    for i, (model_name, results) in enumerate(sorted_models, 1):
        print(f"  {i}. {model_name:15s} | ROC AUC: {results['roc_auc']:.3f} | "
              f"Accuracy: {results['accuracy']:.3f} | F1: {results['f1_score']:.3f}")

# Generate ROC curves and calibration plots
print("\\n🎯 Creating ROC curves and calibration analysis...")
plot_roc_curves_and_calibration(model_results, y_relapse_processed, title_suffix="")

In [ ]:
# Machine learning pipeline for relapse prediction
def train_relapse_prediction_models(X_selected, y_relapse, cv=5):
    """Train multiple ML models specifically for relapse prediction."""
    from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, cross_val_predict
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import classification_report, roc_auc_score
    
    # Define hyperparameter grids
    param_grids = {
        "Random Forest": {
            'n_estimators': [50, 100, 200],
            'max_depth': [5, 10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        "SVM": {
            'C': [0.1, 1, 10],
            'kernel': ['rbf', 'linear'],
            'gamma': ['scale', 'auto']
        },
        "Naive Bayes": {}
    }
    
    # Initialize classifiers
    classifiers = {
        "Random Forest": RandomForestClassifier(random_state=42),
        "SVM": SVC(random_state=42, probability=True),
        "Naive Bayes": GaussianNB()
    }
    
    results = {}
    kf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    
    print(f"🔹 Training models for relapse prediction with {cv}-fold CV 🔹")
    print(f"Dataset: {X_selected.shape[0]} samples, {X_selected.shape[1]} features")
    print(f"Class distribution: {np.bincount(y_relapse)}")
    
    for model_name, model in classifiers.items():
        print(f"\n  Training {model_name}...")
        
        # Hyperparameter tuning
        if param_grids[model_name]:
            grid_search = GridSearchCV(
                model, param_grids[model_name], cv=3, scoring='accuracy', n_jobs=-1
            )
            grid_search.fit(X_selected, y_relapse)
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_
            print(f"    Best params: {best_params}")
        else:
            best_model = model
            best_params = {}
            best_model.fit(X_selected, y_relapse)
        
        # Cross-validation evaluation
        accuracy_scores = cross_val_score(best_model, X_selected, y_relapse, cv=kf, scoring='accuracy')
        roc_auc_scores = cross_val_score(best_model, X_selected, y_relapse, cv=kf, scoring='roc_auc')
        f1_scores = cross_val_score(best_model, X_selected, y_relapse, cv=kf, scoring='f1')
        
        # Generate predictions for detailed analysis
        y_pred = cross_val_predict(best_model, X_selected, y_relapse, cv=kf)
        y_pred_proba = cross_val_predict(best_model, X_selected, y_relapse, cv=kf, method='predict_proba')
        
        # Store results
        results[model_name] = {
            "best_model": best_model,
            "best_params": best_params,
            "accuracy": np.mean(accuracy_scores),
            "accuracy_std": np.std(accuracy_scores),
            "roc_auc": np.mean(roc_auc_scores),
            "roc_auc_std": np.std(roc_auc_scores),
            "f1_score": np.mean(f1_scores),
            "f1_std": np.std(f1_scores),
            "y_pred": y_pred,
            "y_pred_proba": y_pred_proba,
            "classification_report": classification_report(y_relapse, y_pred, output_dict=True)
        }
        
        print(f"    Accuracy: {np.mean(accuracy_scores):.3f} (±{np.std(accuracy_scores):.3f})")
        print(f"    ROC AUC:  {np.mean(roc_auc_scores):.3f} (±{np.std(roc_auc_scores):.3f})")
        print(f"    F1-Score: {np.mean(f1_scores):.3f} (±{np.std(f1_scores):.3f})")
    
    return results

# Train models
print("🚀 Starting relapse prediction model training...")
model_results = train_relapse_prediction_models(X_selected, y_relapse_processed, cv=5)

# Display summary
print(f"\n{'='*60}")
print("🏆 MODEL TRAINING RESULTS SUMMARY")
print(f"{'='*60}")
for model_name, results in model_results.items():
    print(f"{model_name:15s} | Accuracy: {results['accuracy']:.3f} | ROC AUC: {results['roc_auc']:.3f} | F1: {results['f1_score']:.3f}")
print(f"{'='*60}")

In [ ]:
# Feature selection optimization for relapse prediction
def optimize_feature_selection_for_relapse(X, y_relapse, method="select_k_best"):
    """Perform feature selection optimization specifically for relapse prediction."""
    from sklearn.model_selection import cross_val_score
    from sklearn.feature_selection import SelectKBest, f_classif, RFE
    from sklearn.ensemble import RandomForestClassifier
    
    model = RandomForestClassifier(random_state=42, n_estimators=100)
    
    if method == "select_k_best":
        k_values = [1, 2, 3, 5, 8, 13, 21, 34]
        k_values = [k for k in k_values if k <= X.shape[1]]
        
        best_k = k_values[0]
        best_score = -np.inf
        best_selector = None
        
        print(f"Testing k values: {k_values}")
        
        for k in k_values:
            selector = SelectKBest(score_func=f_classif, k=k)
            X_selected = selector.fit_transform(X, y_relapse)
            
            scores = cross_val_score(model, X_selected, y_relapse, cv=5, scoring='accuracy')
            score = np.mean(scores)
            
            print(f"k={k}: accuracy={score:.3f} (±{np.std(scores):.3f})")
            
            if score > best_score:
                best_k, best_score = k, score
                best_selector = selector
        
        selected_features = X.columns[best_selector.get_support()].tolist()
        print(f"\n✅ Optimal k={best_k}, score={best_score:.3f}")
        print(f"Selected features: {selected_features}")
        
        return best_selector.transform(X), selected_features, best_k
    
    elif method == "rfe":
        n_features = [1, 2, 3, 5, 8, 13, 21]
        n_features = [n for n in n_features if n <= X.shape[1]]
        
        best_n = n_features[0]
        best_score = -np.inf
        best_selector = None
        
        for n in n_features:
            selector = RFE(model, n_features_to_select=n)
            X_selected = selector.fit_transform(X, y_relapse)
            
            scores = cross_val_score(model, X_selected, y_relapse, cv=5, scoring='accuracy')
            score = np.mean(scores)
            
            if score > best_score:
                best_n, best_score = n, score
                best_selector = selector
        
        selected_features = X.columns[best_selector.get_support()].tolist()
        return best_selector.transform(X), selected_features, best_n

def create_feature_sets_for_relapse():
    """Define feature sets specifically for relapse prediction."""
    feature_sets = {
        "dynamics_only": {
            "name": "Chimerism Dynamics Only",
            "features": ["d(30-60)_cd3+", "d(60-100)_cd3+", "d(30-60)_cd3-", "d(60-100)_cd3-"]
        },
        "timepoints_only": {
            "name": "Time Points Only", 
            "features": ["d30_cd3+", "d30_cd3-", "d60_cd3+", "d60_cd3-", "d100_cd3+", "d100_cd3-"]
        },
        "statistics_only": {
            "name": "Statistical Features Only",
            "features": ["mean_cd3+", "mean_cd3-", "std_cd3+", "std_cd3-", "cv_cd3+", "cv_cd3-"]
        },
        "day100_focus": {
            "name": "Day 100 Focus",
            "features": ["d100_cd3+", "d100_cd3-", "d(60-100)_cd3+", "d(60-100)_cd3-"]
        },
        "comprehensive": {
            "name": "Comprehensive Chimerism",
            "features": ["d30_cd3+", "d60_cd3+", "d100_cd3+", "d30_cd3-", "d60_cd3-", "d100_cd3-",
                        "d(30-60)_cd3+", "d(60-100)_cd3+", "d(30-60)_cd3-", "d(60-100)_cd3-",
                        "mean_cd3+", "std_cd3+", "cv_cd3+", "mean_cd3-", "std_cd3-", "cv_cd3-"]
        },
        "minimal_predictive": {
            "name": "Minimal Predictive Set",
            "features": ["d(60-100)_cd3+", "d100_cd3-", "std_cd3+"]  # Based on unified analysis results
        }
    }
    
    return feature_sets

# Perform feature selection
print("🔍 Performing feature selection for relapse prediction...")
X_selected, selected_features, optimal_k = optimize_feature_selection_for_relapse(
    X_preprocessed, y_relapse_processed, method="select_k_best"
)

print(f"\nOptimal feature selection completed:")
print(f"Original features: {X_preprocessed.shape[1]}")
print(f"Selected features: {len(selected_features)}")
print(f"Selected feature names: {selected_features}")

In [ ]:
# Feature extraction and preprocessing for relapse prediction only
def extract_features_and_relapse_target(df):
    """Extract features and relapse target specifically."""
    # Filter for AML/MDS patients (Disease == 1)
    if 'disease' in df.columns:
        df_filtered = df[df['disease'] == 1].copy()
        print(f"👥 Filtered to AML/MDS patients: {len(df_filtered)} from {len(df)}")
    else:
        print("⚠️ 'disease' column not found, using all patients")
        df_filtered = df.copy()
    
    # Identify feature columns (exclude outcome variables)
    exclude_cols = ['dose_dli', 'dose_dli_2', 'indication_for_dli', 'post_dli_gvhd',
                   'grade_at_onset', 'time_to_onset', 'highest_grade', 'dli']
    
    feature_cols = [col for col in df_filtered.columns 
                   if not col.startswith('y_') and col not in exclude_cols]
    
    # Extract features and relapse target
    X = df_filtered[feature_cols].copy()
    y_relapse = df_filtered['y_relapse'].copy() if 'y_relapse' in df_filtered.columns else None
    
    if y_relapse is None:
        raise ValueError("y_relapse column not found in dataset!")
    
    # Reset indices
    X.reset_index(drop=True, inplace=True)
    y_relapse.reset_index(drop=True, inplace=True)
    
    print(f"Features shape: {X.shape}")
    print(f"Relapse target shape: {y_relapse.shape}")
    print(f"Relapse distribution: {y_relapse.value_counts().to_dict()}")
    
    return X, y_relapse

def preprocess_features(X, y_relapse):
    """Apply preprocessing specific to relapse prediction."""
    print("🔧 Applying preprocessing...")
    
    X_processed = X.copy()
    
    # Handle categorical variables
    categorical_cols = X_processed.select_dtypes(include=['object', 'category']).columns
    if len(categorical_cols) > 0:
        print(f"🏷️ Encoding {len(categorical_cols)} categorical columns")
        for col in categorical_cols:
            le = LabelEncoder()
            X_processed[col] = le.fit_transform(X_processed[col].astype(str))
    
    # Imputation for numerical columns
    numerical_cols = X_processed.select_dtypes(include=['float64', 'int64']).columns.tolist()
    
    if numerical_cols:
        print(f"📊 Imputing {len(numerical_cols)} numerical columns")
        imputer = SimpleImputer(strategy='median')
        X_processed[numerical_cols] = imputer.fit_transform(X_processed[numerical_cols])
    
    # Scale features
    print("📏 Scaling features")
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(
        scaler.fit_transform(X_processed),
        columns=X_processed.columns,
        index=X_processed.index
    )
    
    # Ensure y_relapse is binary
    y_processed = pd.to_numeric(y_relapse, errors='coerce')
    y_processed = y_processed.fillna(0).astype(int)
    
    print(f"✅ Preprocessing completed")
    print(f"Remaining missing values: {X_scaled.isnull().sum().sum()}")
    
    return X_scaled, y_processed, scaler

# Extract and preprocess data
X, y_relapse = extract_features_and_relapse_target(df_enhanced)
X_preprocessed, y_relapse_processed, scaler = preprocess_features(X, y_relapse)

In [ ]:
# Data loading and basic feature engineering (adapted from unified notebook)
def load_data(file_path="main_dataset.xlsx", sheet_name="Sheet1", skip_rows=2):
    """Load dataset from Excel file with proper handling of headers and empty rows."""
    logging.info("Loading dataset...")
    
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skip_rows)
    except FileNotFoundError:
        logging.info("Excel file not found, trying preprocessed CSV...")
        df = pd.read_csv("preprocessed_ml_for_aml_mds.csv")
    
    df.dropna(axis=0, how='all', inplace=True)
    logging.info(f"Dataset loaded successfully. Shape: {df.shape}")
    return df

def create_chimerism_dynamics_features(df):
    """Engineer comprehensive chimerism dynamics features."""
    print("🔧 Engineering chimerism dynamics features...")
    df_enhanced = df.copy()
    
    required_cols = ['d30_cd3+', 'd60_cd3+', 'd100_cd3+', 'd30_cd3-', 'd60_cd3-', 'd100_cd3-']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        print(f"⚠️ Missing required columns: {missing_cols}")
        print(f"Available columns that contain 'cd3': {[col for col in df.columns if 'cd3' in col.lower()]}")
        return df_enhanced
    
    # Convert to numeric
    for col in required_cols:
        df_enhanced[col] = pd.to_numeric(df_enhanced[col], errors='coerce')
    
    # Time-point differences
    df_enhanced["d(30-60)_cd3+"] = df_enhanced["d30_cd3+"] - df_enhanced["d60_cd3+"]
    df_enhanced["d(60-100)_cd3+"] = df_enhanced["d60_cd3+"] - df_enhanced["d100_cd3+"]
    df_enhanced["d(30-60)_cd3-"] = df_enhanced["d30_cd3-"] - df_enhanced["d60_cd3-"]
    df_enhanced["d(60-100)_cd3-"] = df_enhanced["d60_cd3-"] - df_enhanced["d100_cd3-"]
    df_enhanced["d(30-100)_cd3+"] = df_enhanced["d30_cd3+"] - df_enhanced["d100_cd3+"]
    df_enhanced["d(30-100)_cd3-"] = df_enhanced["d30_cd3-"] - df_enhanced["d100_cd3-"]
    
    # Statistical features
    cd3_pos_cols = ["d30_cd3+", "d60_cd3+", "d100_cd3+"]
    cd3_neg_cols = ["d30_cd3-", "d60_cd3-", "d100_cd3-"]
    
    df_enhanced["mean_cd3+"] = df_enhanced[cd3_pos_cols].mean(axis=1)
    df_enhanced["std_cd3+"] = df_enhanced[cd3_pos_cols].std(axis=1)
    df_enhanced["cv_cd3+"] = df_enhanced["std_cd3+"] / (df_enhanced["mean_cd3+"] + 0.001)
    
    df_enhanced["mean_cd3-"] = df_enhanced[cd3_neg_cols].mean(axis=1)
    df_enhanced["std_cd3-"] = df_enhanced[cd3_neg_cols].std(axis=1)
    df_enhanced["cv_cd3-"] = df_enhanced["std_cd3-"] / (df_enhanced["mean_cd3-"] + 0.001)
    
    print(f"✅ Feature engineering completed. Added {len(df_enhanced.columns) - len(df.columns)} new features.")
    return df_enhanced

# Load and prepare data
df = load_data(file_path="preprocessed_ml_for_aml_mds.csv")
df_enhanced = create_chimerism_dynamics_features(df)

print(f"Dataset shape: {df_enhanced.shape}")
print(f"Relapse target distribution:")
if 'y_relapse' in df_enhanced.columns:
    print(df_enhanced['y_relapse'].value_counts())
else:
    print("y_relapse column not found!")

# Relapse Prediction Analysis

## Comprehensive Analysis of CD3+ Chimerism Dynamics and Relapse Prediction

This notebook focuses specifically on predicting disease relapse using CD3+ chimerism dynamics and other clinical features in AML/MDS patients post-transplant.

### Analysis Framework:
1. **Data Loading and Feature Engineering**: Enhanced chimerism dynamics features
2. **Feature Selection Optimization**: Multiple methods for optimal feature selection
3. **Machine Learning Pipeline**: Classification models with cross-validation
4. **ROC Curves and Calibration Analysis**: Performance evaluation and probability calibration
5. **Clinical Translation**: Risk prediction models and feature importance analysis

In [ ]:
# Core data processing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import logging
import sys
import os
import joblib
import glob
from datetime import datetime

# Machine learning libraries  
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error, mean_squared_error, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, RFE
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc, calibration_curve, brier_score_loss, precision_recall_curve, average_precision_score

# Configuration
warnings.filterwarnings("ignore")
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s", force=True)
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully for relapse prediction analysis")
print(f"📅 Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")